# Load Libraries

In [1]:
import numpy as np
import pandas as pd
import json
from pathlib import Path
import re

## Load tag data

In [2]:
file = r'./domain_instances.xls'
tagDF = pd.read_excel(file)

In [3]:
tagDF.head()

,Environment,Domain,Host,LOB,Portfolio,Product,Application,Owner
0,DEV0,ASSIGNMENT,assignmentadm001.da.aws.cccis.com,ISG,Workflow,Claims Management,NaN,build
1,DEV0,ASSIGNMENT,assignmentodrp001.da.aws.cccis.com,ISG,Workflow,Claims Management,NaN,build
2,DEV0,ASSIGNMENT,assignmentodrp002.da.aws.cccis.com,ISG,Workflow,Claims Management,NaN,build
3,DEV0,ASSIGNMENT,assignmentappraiser001.da.aws.cccis.com,ISG,Workflow,Appraiser Management,NaN,build
4,DEV0,ASSIGNMENT,assignmentappraiser002.da.aws.cccis.com,ISG,Workflow,Appraiser Management,NaN,build


In [4]:
len(tagDF.Host.unique())

934

## Load instance data

In [5]:
file = r'./domain_volume.xlsm'
instanceDF = pd.read_excel(file)

In [6]:
instanceDF.head()

,Instance_id,Instance_name,Volume_id,LOB,Portfolio,Product,Application,Owner,Unnamed: 8,Unnamed: 9,...,Unnamed: 16374,Unnamed: 16375,Unnamed: 16376,Unnamed: 16377,Unnamed: 16378,Unnamed: 16379,Unnamed: 16380,Unnamed: 16381,Unnamed: 16382,Unnamed: 16383
0,i-f982d879,intrnladm001.da.aws.cccis.com,vol-14006bcb,Enterprise Services,Shared Services,Messaging,Internal Email,build,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,i-0c44ab2f63f3ddda7,RHEL 7.3 Mini Test,vol-069027e027c4820e8,Enterprise Services,Infrastructure,Systems Management,Back up,aptashnik,modified,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,i-02bfb7e7e1766d267,elk-cccis-ls-001,vol-0680f23d605869a20,Enterprise Services,Infrastructure,Monitoring,ELK,rkrishnamurthy,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,i-02bfb7e7e1766d267,elk-cccis-ls-001,vol-09b2c79fd72a926f1,Enterprise Services,Infrastructure,Monitoring,ELK,rkrishnamurthy,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,i-02765b0493f4e8f39,Hudson-slave-001,vol-0d3455bd7dcd8263f,Enterprise Services,Infrastructure,Build Management,Hudson,rkrishnamurthy,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
len(instanceDF.Instance_id.unique())

1446

## Load ami data

In [8]:
file = r'./ec2_ami.csv'
amiDF = pd.read_csv(file, header=None)
amiDF.columns = ['i_id', 'ami']

In [9]:
amiDF.head()

,i_id,ami
0,i-0c908b1552a684ef7,ami-6374af08
1,i-6ed4c4d1,ami-12ebbd7a
2,i-04ae568770a61d549,ami-722fe11f
3,i-048781e21c711a091,ami-1f7b2a08
4,i-afbafa0c,ami-b0210ed8


In [10]:
len(amiDF.i_id.unique())

1449

## Create DF for instance info

In [11]:
# of the 934 instances for our analysis (in the tag data)
# id, host (instance name), ami

# merge tag data with instance data (obtain host name and id)
merged = pd.merge(tagDF, instanceDF, how='inner', left_on=['Host'], right_on=['Instance_name'])
host_id = merged[['Host', 'Instance_id']]

In [12]:
host_id.head()

,Host,Instance_id
0,assignmentadm001.da.aws.cccis.com,i-04b7e0ef13ecdba59
1,assignmentodrp001.da.aws.cccis.com,i-0b947ccb2dce8a044
2,assignmentodrp002.da.aws.cccis.com,i-036d5c00f77ba5fa0
3,assignmentappraiser001.da.aws.cccis.com,i-0ac3524dde42e2a12
4,assignmentappraiser002.da.aws.cccis.com,i-0abd88a93f2a55b1e


In [13]:
len(host_id)

940

In [14]:
len(host_id.Host.unique())

926

In [15]:
# keep only unique 
host_id = host_id.drop_duplicates()

In [16]:
len(host_id)

926

In [17]:
# merge with ami 

merged2 = pd.merge(host_id, amiDF, how='inner', left_on=['Instance_id'], right_on=['i_id'])
i_ami = merged2.drop(['i_id'], axis=1)

In [18]:
i_ami.head()

,Host,Instance_id,ami
0,assignmentadm001.da.aws.cccis.com,i-04b7e0ef13ecdba59,ami-722fe11f
1,assignmentodrp001.da.aws.cccis.com,i-0b947ccb2dce8a044,ami-722fe11f
2,assignmentodrp002.da.aws.cccis.com,i-036d5c00f77ba5fa0,ami-722fe11f
3,assignmentappraiser001.da.aws.cccis.com,i-0ac3524dde42e2a12,ami-722fe11f
4,assignmentappraiser002.da.aws.cccis.com,i-0abd88a93f2a55b1e,ami-722fe11f


In [19]:
len(i_ami.Host.unique())

926

In [20]:
len(i_ami.ami.unique())

16

In [21]:
len(i_ami)

926

## Save Instance info Dataframe

In [22]:
i_ami.to_csv('./dataframes/instanceData.csv',sep=',',index=False, encoding="utf-8")